In [4]:
# REQUIREMENTS
! pip install ipyleaflet
! pip install bs4
! pip install requests


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.3 MB 5.0 MB/s 


In [52]:
#Import libraries
from ipyleaflet import Map, basemaps, basemap_to_tiles, TileLayer
from bs4 import BeautifulSoup
import requests
import ast #Abstract syntax trees 


### Scrape all available URLS using Beautiful Soup


In [40]:
#This URL can be changed
url='https://mapcore-demo.org/current/flatmap/v2/'
req= requests.get(url)
content=req.text
url_dict = ast.literal_eval(content)

In [50]:
#JOB : There are fields with non IDs, len=6 eliminated this, think of a smarter way!
model_ids = [model_dict['id'] for model_dict in url_dict if len(model_dict) == 6]
model_name = [model_dict['name'] for model_dict in url_dict if len(model_dict) == 6]
  

### Get all layers for each model to be added sequentially to the map

In [75]:
model_layer_url_list = ['https://mapcore-demo.org/current/flatmap/v2/flatmap/'+id+'/layers'   for id, name in zip(model_ids, model_name)]

In [98]:
model_image_layers = []
for url in model_layer_url_list:
  req= requests.get(url)
  content=req.text
  layer_url_dict = ast.literal_eval(content)
  model_image_layers.append({layer_url_dict[0]['id'] : layer_url_dict[0]['image-layers']})



In [122]:
model_image_layers[1]

{'whole-rat': ['whole-rat_image',
  'whole-rat_details_vagus_image',
  'whole-rat_details_tissue-slide_image']}

### Create a map with all layers of a model added seqentially to a map

In [134]:
#Choose one of the models in tag (There are 11 in the main URL)
tag = 2
tile_urls = ['https://mapcore-demo.org/current/flatmap/v2/flatmap/' + model_ids[tag] + '/tiles/' + layer + '/{z}/{x}/{y}' for layer in model_image_layers[tag][model_name[tag]]]
tile_urls


['https://mapcore-demo.org/current/flatmap/v2/flatmap/1220ab6b172448ccf9dd8c4d748447248d3185b25123dd5b4700d33c04d80011/tiles/whole-pig_image/{z}/{x}/{y}']

In [135]:
#Create a base map object with the first tile
map = Map(
    basemap=TileLayer(
                    url=tile_urls[0], 
                    min_zoom=0),
    min_zoom=0,
    zoom=0,
    scroll_wheel_zoom=True,
    dragging=True,
    attribution_control=False,
    zoom_snap=False,
)

#Create tile objects for the model with each layer added
for tile_url in tile_urls[1:]:

  map.add(TileLayer(
    url=tile_url, 
    min_zoom=0))

### Add annotations for the map

In [137]:
model_annotation_url_list = ['https://mapcore-demo.org/current/flatmap/v2/flatmap/'+id+'/annotations'   for id, name in zip(model_ids, model_name)]

In [138]:
model_annotation_url_list

['https://mapcore-demo.org/current/flatmap/v2/flatmap/4e758a1feb4dd73ac500b29dcf65e9c80031395fd9c2ee4c521c2e784c9885fc/annotations',
 'https://mapcore-demo.org/current/flatmap/v2/flatmap/e2b09706ce736b49d48f49f7c83296f1a9e905d26a247da3e7854fb9f2bbdda5/annotations',
 'https://mapcore-demo.org/current/flatmap/v2/flatmap/1220ab6b172448ccf9dd8c4d748447248d3185b25123dd5b4700d33c04d80011/annotations',
 'https://mapcore-demo.org/current/flatmap/v2/flatmap/f28ef9e4e6bc0ab59b074f82944b9238dad70b37fcf65cc9773089f10d75831a/annotations',
 'https://mapcore-demo.org/current/flatmap/v2/flatmap/f94da5b45b2b33f420e51a3663056a9d98a7567b24eaf62fd77a347f97c4cf5a/annotations',
 'https://mapcore-demo.org/current/flatmap/v2/flatmap/1340312b715b51552a0a62666b3ae4e7ccd40276e359d83d72dbe285672faf5e/annotations',
 'https://mapcore-demo.org/current/flatmap/v2/flatmap/a63ea82480f916f293443c290a32813db60b11acddacadf23a5effff9b150df8/annotations',
 'https://mapcore-demo.org/current/flatmap/v2/flatmap/66daf96873d852f

In [ ]:
model_image_annotations = []
for url in model_annotation_url_list[1]:
  req= requests.get(url)
  content=req.text
  #annot_url_dict = ast.literal_eval(content)
  print(content)
  #model_image_annotations.append({layer_url_dict[0]['id'] : layer_url_dict[0]['image-layers']})

In [145]:
  #Error is that literal_eval encountered a decimal in coordinates and is not happy with it; it has to be a string.
  req= requests.get('https://mapcore-demo.org/current/flatmap/v2/flatmap/e2b09706ce736b49d48f49f7c83296f1a9e905d26a247da3e7854fb9f2bbdda5/annotations')
  content=req.text
  annot_url_dict = ast.literal_eval(content)

ValueError: ignored

In [146]:
content

'{"1":{"bounds":[4.5535908228057105,-5.3089305257115464,6.014318708488964,-3.7926937100778564],"centroid":[5.162284208458908,-4.438206330299932],"featureId":1,"geometry":"Polygon","layer":"whole-rat","tile-layer":"features"},"100":{"bounds":[-3.444293996750576,-3.665713653468196,-2.9302900206530293,-3.1476957768091127],"centroid":[-3.1531061665935853,-3.4429536285341045],"class":"brain_26","featureId":100,"geometry":"Polygon","label":"pons","layer":"whole-rat","markup":".region class(brain_26)","models":"UBERON:0000988","region":true,"tile-layer":"features"},"101":{"bounds":[-2.93842289134658,-3.644379722416018,-2.239119244403009,-3.138507758881123],"centroid":[-2.621446836178815,-3.3823001007200655],"class":"brain_27","featureId":101,"geometry":"Polygon","label":"medulla oblongata","layer":"whole-rat","markup":".region class(brain_27)","models":"UBERON:0001896","region":true,"tile-layer":"features"},"1018":{"bounds":[-2.6230632030002607,7.525316823735705,-2.158698729807565,7.938397982